# Example 4 - Plot echogram with PyEcholab

In this example, we'll use PyEcholab to plot some example echograms in 2019. 

In [28]:
import os
import glob
import numpy as np
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pickle
import pandas as pd

from echolab2.instruments import EK60
from echolab2.processing import align_pings
from echolab2.processing.mask import Mask
from echolab2.processing.line import Line
from echolab2.plotting.matplotlib import echogram
from echolab2.processing import mask, line
import matplotlib.patches as patches

from src.read_echogram_plotting import EchogramReader

from sklearn.preprocessing import StandardScaler, MinMaxScaler

sns.set(font_scale=1.2)
plt.style.use('seaborn-white')
%matplotlib inline

## Step 1. Plot echogram only

In this step, we'll plot echogram only, without annotations. 

In [2]:
test_examples = ['D20190925-T204502', 'D20190926-T230434', 'D20191001-T065053', 'D20191001-T085625', 'D20191001-T155448', 'D20191002-T112235', 'D20191015-T203008', 'D20190927-T072325', 'D20191016-T184753', 'D20191016-T213424', 'D20191018-T081659', 'D20191018-T110329', 'D20191020-T145420', 'D20191024-T103607', 'D20191024-T172924', 'D20191102-T144417', 'D20191102-T160647'] # 10

In [3]:
# add raw and bottom file dir
raw_dir = "../data/HB1906_EK60/rawfiles/"
bot_dir = "../data/HB1906_EK60/botfiles/"
freq_li = [18, 38, 120, 200]

In [4]:
# add fig dir
fig_dir = "figures/"

In [10]:
for item in test_examples:
    raw_path = os.path.join(raw_dir, item + '.raw')
    bot_path = os.path.join(bot_dir, item + '.bot')
    new_echogram = EchogramReader(raw_path, bot_path, freq_li)
    filename, Sv_data, interp, depth = new_echogram() 
    plot_depth = np.max(interp) + 5 # set maximum depth
    # plot
    fig = plt.figure(figsize=[25, 15])
    ax = fig.add_subplot(111)
    eg = echogram.Echogram(ax, Sv_data[1], threshold=[-66, 0]) # use -66dB as threshold
    xmin, xmax, ymin, ymax = plt.axis()
    ax.grid(False)
    plt.ylim((depth[0], plot_depth)) 
    plt.gca().invert_yaxis()
    plt.ylabel('Depth (m)', fontsize=50)
    plt.xlabel(filename, fontsize=50)
    plt.yticks(fontsize=40)
    plt.xticks(fontsize=40)
    plt.tight_layout(pad=0.1)
    plt.savefig(fig_dir + filename + '.png', dpi=300)
    plt.close()

D20190925-T204502
D20190926-T230434
D20191001-T065053
D20191001-T085625
D20191001-T155448
D20191002-T112235
D20191015-T203008
D20190927-T072325
D20191016-T184753
D20191016-T213424
D20191018-T081659
D20191018-T110329
D20191020-T145420
D20191024-T103607
D20191024-T172924
D20191102-T144417
D20191102-T160647


## Step 2. Draw output of each step (for framework)

In this step, we'll draw output of each step for framework. 

In [5]:
item = 'D20191024-T103607'

Draw original image. 

In [6]:
raw_path = os.path.join(raw_dir, item + '.raw')
bot_path = os.path.join(bot_dir, item + '.bot')
new_echogram = EchogramReader(raw_path, bot_path, freq_li)
filename, Sv_data, interp, depth = new_echogram() 
plot_depth = np.max(interp) + 10 # set maximum depth
# plot
fig = plt.figure(figsize=[25, 15])
ax = fig.add_subplot(111)
eg = echogram.Echogram(ax, Sv_data[1], threshold=[-80, 0]) # use -80dB as threshold
xmin, xmax, ymin, ymax = plt.axis()
ax.grid(False)
plt.ylim((depth[0], plot_depth)) 
plt.gca().invert_yaxis()
plt.ylabel('Depth (m)', fontsize=50)
plt.xlabel(filename, fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(fontsize=40)
plt.tight_layout(pad=0.1)
plt.savefig(fig_dir + 'origin_image.png', dpi=300)
plt.close()

D20191024-T103607


Draw contour image. 

In [18]:
# load
from src.read_echogram import EchogramReader
from src.detect_ROI import ROIDetector
from src.ROI_features import FeatureExtractor

In [13]:
threshold = -66
kernel_size = 3

In [19]:
# detect ROI
echogram = EchogramReader(raw_path, bot_path, freq_li)
filename, Sv_npy, surface_idx, bottom_idx, time, depth, positions = echogram()   
roi = ROIDetector(filename, Sv_npy, surface_idx, bottom_idx, fig_dir, threshold, kernel_size)
img_shape, contours = roi()
features = FeatureExtractor(filename, contours, Sv_npy, bottom_idx, time, depth, positions)                    
contours_sel, contours_features = features() 

D20191024-T103607


In [22]:
img = np.zeros([img_shape[0], img_shape[1], 3], dtype=np.int32)
cv2.drawContours(img, contours_sel, -1, (0, 255, 0), thickness=2)
cv2.imwrite(fig_dir + 'contour_image.png', img)

True

Draw object outline, use polygons. 

In [25]:
from src.read_echogram_plotting import EchogramReader

In [27]:
new_echogram = EchogramReader(raw_path, bot_path, freq_li)
filename, Sv_data, interp, depth = new_echogram() 
plot_depth = np.max(interp) + 10 # set maximum depth
# plot
fig = plt.figure(figsize=[25, 15])
ax = fig.add_subplot(111)
eg = echogram.Echogram(ax, Sv_data[1], threshold=[-80, 0]) # use -80dB as threshold
for contour in contours_sel:
    x_li = contour[:, 0, 0] 
    y_li = contour[:, 0, 1] 
    x_li_idx = [((time[i]).astype(datetime) - datetime(1970, 1, 1)).total_seconds() * 1000.0 for i in x_li]
    y_li_idx = [depth[j] for j in y_li]
    xy_li_idx = [[i, j] for i, j in zip(x_li_idx, y_li_idx)]
    ax.add_patch(patches.Polygon(np.array(xy_li_idx), fill=False, linewidth=1.5, edgecolor='orange'))
xmin, xmax, ymin, ymax = plt.axis()
ax.grid(False)
plt.ylim((depth[0], plot_depth)) 
plt.gca().invert_yaxis()
plt.ylabel('Depth (m)', fontsize=50)
plt.xlabel(filename, fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(fontsize=40)
plt.tight_layout(pad=0.1)
plt.savefig(fig_dir + 'classification_image.png', dpi=300)
plt.close()

D20191024-T103607


## Step 3. Draw figures about ROIs and annotations

In this step, we'll show when there are some missing data in annotations.

In [7]:
item = 'D20191102-T144417'

In [8]:
# load
from src.read_echogram import EchogramReader
from src.detect_ROI import ROIDetector
from src.ROI_features import FeatureExtractor

In [9]:
threshold = -66
kernel_size = 3

Draw ROIs first. 

In [11]:
raw_path = os.path.join(raw_dir, item + '.raw')
bot_path = os.path.join(bot_dir, item + '.bot')
echogram = EchogramReader(raw_path, bot_path, freq_li)
filename, Sv_npy, surface_idx, bottom_idx, time, depth, positions = echogram()   
roi = ROIDetector(filename, Sv_npy, surface_idx, bottom_idx, fig_dir, threshold, kernel_size)
img_shape, contours = roi()
features = FeatureExtractor(filename, contours, Sv_npy, bottom_idx, time, depth, positions)                    
contours_sel, contours_features = features() 

D20191102-T144417


In [12]:
img = np.zeros([img_shape[0], img_shape[1], 3], dtype=np.int32)
cv2.drawContours(img, contours_sel, -1, (0, 255, 0), thickness=2)
cv2.imwrite(fig_dir + 'D20191102-T144417_ROI.png', img)

True

Then, let's draw annotations. 

In [19]:
from src.read_echogram_plotting import EchogramReader
import pickle

In [24]:
new_echogram = EchogramReaderPlot(raw_path, bot_path, freq_li)
filename, Sv_data, interp, depth = new_echogram() 
plot_depth = np.max(interp) + 10 # set maximum depth
# plot
fig = plt.figure(figsize=[25, 15])
ax = fig.add_subplot(111)
eg = echogram.Echogram(ax, Sv_data[1], threshold=[-80, 0]) # use -80dB as threshold
# get annotations
pkl_dir = "pkl/"
with open(pkl_dir + 'annotations_dict_new.pickle', 'rb') as handle:
    annotations_dict = pickle.load(handle)
annotations_idx, labels = annotations_dict[item]
annotations_sel = annotations_idx[[idx for idx, i in enumerate(labels) if i == 4]]
for contour in annotations_sel:
    x_li = contour[:, 0, 0] 
    y_li = contour[:, 0, 1] 
    x_li_idx = [((time[i]).astype(datetime) - datetime(1970, 1, 1)).total_seconds() * 1000.0 for i in x_li]
    y_li_idx = [depth[j] for j in y_li]
    xy_li_idx = [[i, j] for i, j in zip(x_li_idx, y_li_idx)]
    ax.add_patch(patches.Polygon(np.array(xy_li_idx), fill=False, linewidth=1.5, edgecolor='orange'))
xmin, xmax, ymin, ymax = plt.axis()
ax.grid(False)
plt.ylim((depth[0], plot_depth)) 
plt.gca().invert_yaxis()
plt.ylabel('', fontsize=50)
plt.xlabel('', fontsize=50)
plt.xticks([])
plt.yticks([])
# plt.yticks(fontsize=40)
# plt.xticks(fontsize=40)
plt.tight_layout(pad=0.1)
plt.savefig(fig_dir + 'D20191102-T144417_annotations.png', dpi=300)
plt.close()

D20191102-T144417


## Step 4. Prediction example

In this step, we'll use two examples to show how the whole framework would be applied for prediction of Atlantic herring school in unseen echograms. 

In [279]:
item = 'D20191023-T130820'
raw_path = os.path.join(raw_dir, item + '.raw')
bot_path = os.path.join(bot_dir, item + '.bot')

In [280]:
threshold = -66
kernel_size = 1

Draw annotations first. 

In [281]:
# load
from src.read_echogram import EchogramReader
from src.detect_ROI import ROIDetector
from src.ROI_features import FeatureExtractor

In [282]:
raw_path = os.path.join(raw_dir, item + '.raw')
bot_path = os.path.join(bot_dir, item + '.bot')
echogram = EchogramReader(raw_path, bot_path, freq_li)
filename, Sv_npy, surface_idx, bottom_idx, time, depth, positions = echogram()   
roi = ROIDetector(filename, Sv_npy, surface_idx, bottom_idx, fig_dir, threshold, kernel_size)
img_shape, contours = roi()
features = FeatureExtractor(filename, contours, Sv_npy, bottom_idx, time, depth, positions)                    
contours_sel, contours_features = features() 

D20191023-T130820


In [283]:
len(contours_sel)

85

In [285]:
from src.read_echogram_plotting import EchogramReader
from echolab2.plotting.matplotlib import echogram

In [288]:
new_echogram = EchogramReader(raw_path, bot_path, freq_li)
filename, Sv_data, interp, depth = new_echogram() 
plot_depth = np.max(interp) + 10 # set maximum depth
# plot
fig = plt.figure(figsize=[25, 15])
ax = fig.add_subplot(111)
eg = echogram.Echogram(ax, Sv_data[1], threshold=[-80, 0]) # use -80dB as threshold
# get annotations
pkl_dir = "pkl/"
with open(pkl_dir + 'annotations_dict_new.pickle', 'rb') as handle:
    annotations_dict = pickle.load(handle)
annotations_idx, labels = annotations_dict[item]
# annotations_sel = annotations_idx[[idx for idx, i in enumerate(labels) if i == 4]]
annotations_sel = annotations_idx
for idx, contour in enumerate(annotations_sel):
    x_li = contour[:, 0, 0] 
    y_li = contour[:, 0, 1] 
    x_li_idx = [((time[i]).astype(datetime) - datetime(1970, 1, 1)).total_seconds() * 1000.0 for i in x_li]
    y_li_idx = [depth[j] for j in y_li]
    xy_li_idx = [[i, j] for i, j in zip(x_li_idx, y_li_idx)]
    if labels[idx] == 4:
        ax.add_patch(patches.Polygon(np.array(xy_li_idx), fill=False, linewidth=1.5, edgecolor='orange'))
    if labels[idx] == 3:
        ax.add_patch(patches.Polygon(np.array(xy_li_idx), fill=False, linewidth=1.5, edgecolor='pink'))
    if labels[idx] == 1:
        ax.add_patch(patches.Polygon(np.array(xy_li_idx), fill=False, linewidth=1.5, edgecolor='darkgreen'))
xmin, xmax, ymin, ymax = plt.axis()
ax.grid(False)
plt.ylim((depth[0], plot_depth)) 
plt.gca().invert_yaxis()
plt.ylabel('Depth (m)', fontsize=50)
plt.xlabel(filename, fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(fontsize=40)
plt.tight_layout(pad=0.1)
plt.savefig(fig_dir + f'example_7_annotations.png', dpi=300)
plt.close()

D20191023-T130820


In [251]:
# load classifier
with open(pkl_dir + 'model_CO.pkl', 'rb') as handle:
    c = pickle.load(handle)

In [252]:
# load ROI features
pkl_dir = "pkl/"
df_roi_features = pd.read_pickle(pkl_dir + 'df_roi_features_new.pkl')

In [253]:
acoustic_features = ['Sv_18kHz_min', 'Sv_18kHz_p5', 'Sv_18kHz_p25', 'Sv_18kHz_p50', 'Sv_18kHz_p75', 'Sv_18kHz_p95', 'Sv_18kHz_max', 'Sv_18kHz_std', 'Sv_38kHz_min', 'Sv_38kHz_p5', 'Sv_38kHz_p25', 'Sv_38kHz_p50', 'Sv_38kHz_p75', 'Sv_38kHz_p95', 'Sv_38kHz_max', 'Sv_38kHz_std', 'Sv_120kHz_min', 'Sv_120kHz_p5', 'Sv_120kHz_p25', 'Sv_120kHz_p50', 'Sv_120kHz_p75', 'Sv_120kHz_p95', 'Sv_120kHz_max', 'Sv_120kHz_std', 'Sv_200kHz_min', 'Sv_200kHz_p5', 'Sv_200kHz_p25', 'Sv_200kHz_p50', 'Sv_200kHz_p75', 'Sv_200kHz_p95', 'Sv_200kHz_max', 'Sv_200kHz_std', 'Sv_ref_18kHz', 'Sv_ref_120kHz', 'Sv_ref_200kHz']
geometric_features = ['length', 'thickness', 'area', 'perimeter', 'rectangularity', 'compact', 'circularity', 'elongation']
geographic_features_vertical = ['total_water_column', 'depth', 'relative_altitude']
geographic_features_horizontal = ['latitude', 'longitude']
sel_features = acoustic_features + geometric_features + geographic_features_vertical + geographic_features_horizontal

In [254]:
len(acoustic_features), len(geometric_features)

(35, 8)

In [255]:
# normalization
min_max_scaler = MinMaxScaler()
df_roi_features[sel_features] = min_max_scaler.fit_transform(df_roi_features[sel_features])

In [256]:
test_set = df_roi_features[df_roi_features['filename']==item]
X_test = test_set[sel_features].to_numpy()
X1_test = test_set[acoustic_features + geometric_features].to_numpy()
X2_test = test_set[geographic_features_horizontal + geographic_features_vertical].to_numpy()
c_y_predict = c.predict(X1_test, X2_test)

In [257]:
np.unique(c_y_predict, return_counts=True)

(array([0, 1]), array([ 2, 83]))

In [258]:
contours_sel_sel = []
for idx, i in enumerate(c_y_predict):
    if i == 1:
        contours_sel_sel.append(contours_sel[idx])

In [259]:
new_echogram = EchogramReader(raw_path, bot_path, freq_li)
filename, Sv_data, interp, depth = new_echogram() 
plot_depth = np.max(interp) + 10 # set maximum depth
# plot
fig = plt.figure(figsize=[25, 15])
ax = fig.add_subplot(111)
eg = echogram.Echogram(ax, Sv_data[1], threshold=[-80, 0]) # use -80dB as threshold
for contour in contours_sel_sel:
    x_li = contour[:, 0, 0] 
    y_li = contour[:, 0, 1] 
    x_li_idx = [((time[i]).astype(datetime) - datetime(1970, 1, 1)).total_seconds() * 1000.0 for i in x_li]
    y_li_idx = [depth[j] for j in y_li]
    xy_li_idx = [[i, j] for i, j in zip(x_li_idx, y_li_idx)]
    ax.add_patch(patches.Polygon(np.array(xy_li_idx), fill=False, linewidth=1.5, edgecolor='orange'))
xmin, xmax, ymin, ymax = plt.axis()
ax.grid(False)
plt.ylim((depth[0], plot_depth)) 
plt.gca().invert_yaxis()
plt.ylabel('Depth (m)', fontsize=50)
plt.xlabel(filename, fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(fontsize=40)
plt.tight_layout(pad=0.1)
plt.savefig(fig_dir + f'example_7_predictions.png', dpi=300)
plt.close()

D20191023-T130820
